# 0.라이브러리 로딩

In [1]:
import os
import numpy as np
import librosa
import torch
import pyworld as pw
import parselmouth
import argparse
import shutil
import subprocess
import soundfile
from logger import utils
from tqdm import tqdm
from glob import glob
from pydub import AudioSegment, effects
from ddsp.vocoder import F0_Extractor, Volume_Extractor, Units_Encoder
from logger.utils import traverse_dir
from preprocess import preprocess
from train import ddsp_train
from draw import main
from sep_wav import demucs, audio_norm, get_ffmpeg_args
from main import inference
# Cuda setting
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# configure loading
args = utils.load_config('./configs/combsub.yaml')

# set path
MP4_DATA_PATH   = 'preprocess/mp4'
ORIGINAL_PATH   = 'preprocess/original/'
DEMUCS_PATH     = 'preprocess/demucs/'
NORM_PATH       = 'preprocess/norm/'
TEMP_LOG_PATH   = 'temp_ffmpeg_log.txt'  # ffmpeg의 무음 감지 로그의 임시 저장 위치

# 1.데이터 전처리
***
1. 난 전처리가 필요없다. 배경음 제거가 완벽하고, 모든 데이터들도 특정 길이로 잘 잘려져있다.
    - 데이터를 전부 data/train/audio/안에 다 집어 넣고
        ```
        # training dataset
        data/train/audio/aaa.wav
        data/train/audio/bbb.wav
        ...
        ```
    - 1.6 validation 분리단계로 점프
***
2. 난 거의 다 되어 있지만 데이터가 너무 길다. 특정 길이로 자르고 싶다.
    - 데이터를 전부 preprocess/split 안에 다 집어 넣고
        ```
        # training dataset
        preprocess/norm/aaa.wav
        preprocess/norm/bbb.wav
        ...
        ```
    - 1.4 split 단계로 점프
***
3. 난 배경음도 제거해야되고 데이터도 길다. 거의 날 것의 상태다.
    - 데이터를 전부 preprocess/original 안에 다 집어넣고
        ```
        # training dataset
        preprocess/original/aaa.wav
        preprocess/original/bbb.wav
        ...
        ```
    - 1.2부터 demucs 단계로 점프
***
4. 난 아무것도 안되어 있고, 심지어 mp4파일이다.
    - 데이터들 전부 preprocess/mp4에 다 집어넣고
        ```
        # training dataset
        preprocess/mp4/aaa.mp4
        preprocess/mp4/bbb.mp4
        ...
        ```
    - 1.1부터 차례대로 진행

## 1.1데이터가 mp4인 경우 (wav만 있는 경우에는 패스)
preprocess/mp4 안에 있는 mp4파일을 wav로 변경 해서 preprocess/original 에 저장

In [30]:
def mp4_to_wav(input_dir : str, input_file: str, output_dir: str):
    """mp4파일을 wav형식으로 변환합니다.
    Args:
        input_dir (str) : 입력 mp4파일의 path
        input_file (str) : 입력 mp4파일의 이름
        output_dir (str) : 출력 wav파일의 path
    """
    ext = os.path.splitext(input_file)[1][1:]

    if ext != "mp4":
        return 
    else :
        track = AudioSegment.from_file(os.path.join(input_dir,input_file),  format= 'mp4')
        track = track.set_frame_rate(44100)
        track.export(os.path.join(output_dir, input_file[:-4]+".wav"), format='wav')


filelist =  traverse_dir(
    MP4_DATA_PATH,
    extension='mp4',
    is_pure=True,
    is_sort=True,
    is_ext=True)

for file in tqdm(filelist):
    mp4_to_wav(MP4_DATA_PATH, file, ORIGINAL_PATH)

100%|██████████| 12/12 [00:02<00:00,  5.13it/s]


## 1.2 무음제거 (Demucs)
preprocess/original에 있는 wav파일들의 음악소리를 제거하고 목소리만 추출해서 preprocess/demucs 에 저장

In [6]:
demucs(ORIGINAL_PATH, DEMUCS_PATH)

2023-04-16 03:28:16 | INFO | torchaudio.utils.download | The local file (C:\Users\wlsdm\.cache\torch\hub\torchaudio\models\hdemucs_high_trained.pt) exists. Skipping the download.


Sample rate: 44100


목소리 추출 중...: 100%|██████████| 12/12 [00:14<00:00,  1.23s/it]


## 1.3 normalize
preprocess/demucs에 있는 배경음이 제거된 데이터들을 노멀라이즈 해서 preprocess/norm에 저장

In [2]:
for filepath in tqdm(glob(DEMUCS_PATH+"*.wav"), desc="노멀라이징 작업 중..."):
    filename = os.path.splitext(os.path.basename(filepath))[0]
    out_filepath = os.path.join(NORM_PATH, filename) + ".wav"
    audio_norm(filepath, out_filepath)

노멀라이징 작업 중...: 100%|██████████| 12/12 [00:07<00:00,  1.52it/s]


## 1.4 split
preprocess/norm에 있는 노말라이즈된 데이터들을 15초 길이로 잘라서 data/train/audio에 저장

In [11]:
for filepath in tqdm(glob(NORM_PATH+"*.wav"), desc="음원 자르는 중..."):
    duration = librosa.get_duration(filename=filepath)
    max_last_seg_duration = 0
    sep_duration_final = 15
    sep_duration = 15

    while sep_duration > 4:
        last_seg_duration = duration % sep_duration
        if max_last_seg_duration < last_seg_duration:
            max_last_seg_duration = last_seg_duration
            sep_duration_final = sep_duration
        sep_duration -= 1

    filename = os.path.splitext(os.path.basename(filepath))[0]
    out_filepath = os.path.join(args.data.train_path,"audio", f"{filename}-%04d.wav")
    subprocess.run(f'ffmpeg -i "{filepath}" -f segment -segment_time {sep_duration_final} "{out_filepath}" -y', capture_output=True, shell=True)


음원 자르는 중...: 100%|██████████| 12/12 [00:00<00:00, 14.80it/s]


## 1.5 무음 제거
data/train/audio에 있는 잘라진 음원들 중에 무음인 파일들을 제거

In [14]:
for filepath in tqdm(glob(args.data.train_path+"/audio/*.wav"), desc="무음 제거 중..."):
    if os.path.exists(TEMP_LOG_PATH):
        os.remove(TEMP_LOG_PATH)

    ffmpeg_arg = get_ffmpeg_args(filepath)
    subprocess.run(ffmpeg_arg, capture_output=True, shell=True)

    start = None
    end = None

    with open(TEMP_LOG_PATH, "r", encoding="utf-8") as f:
        for line in f.readlines():
            line = line.strip()
            if "lavfi.silence_start" in line:
                start = float(line.split("=")[1])
            if "lavfi.silence_end" in line:
                end = float(line.split("=")[1])

    if start != None:
        if start == 0 and end == None:
            os.remove(filepath)
            
if os.path.exists(TEMP_LOG_PATH):
        os.remove(TEMP_LOG_PATH)

무음 제거 중...: 100%|██████████| 108/108 [00:04<00:00, 23.06it/s]


## 1.6 학습 데이터 중, 일부를 validaion용으로 자동으로 보내준다.
- data/train/audio에 있는 데이터 중 일정 비율만큼 알아서 data/val/audio로 이동시켜준다
    - 계산식은 다음과 같다 `max(2, min(10, 전체 데이터 * 0.01))`

In [15]:
main()

100%|██████████| 2/2 [00:00<00:00, 1342.39it/s]


## 2. 데이터 전처리 (학습에 쓰기 위한)

In [16]:
# get data
sample_rate = args.data.sampling_rate
hop_size = args.data.block_size

# initialize f0 extractor
f0_extractor = F0_Extractor(
                    args.data.f0_extractor, 
                    args.data.sampling_rate, 
                    args.data.block_size, 
                    args.data.f0_min, 
                    args.data.f0_max)

# initialize volume extractor
volume_extractor = Volume_Extractor(args.data.block_size)
                    
# initialize units encoder
units_encoder = Units_Encoder(
                    args.data.encoder, 
                    args.data.encoder_ckpt, 
                    args.data.encoder_sample_rate, 
                    args.data.encoder_hop_size, 
                    device = device)    

# preprocess training set
preprocess(args.data.train_path, f0_extractor, volume_extractor, units_encoder, sample_rate, hop_size, device = device)

# preprocess validation set
preprocess(args.data.valid_path, f0_extractor, volume_extractor, units_encoder, sample_rate, hop_size, device = device)

 [Encoder Model] HuBERT Soft
 [Loading] pretrain/hubert/hubert-soft-0d54a1f4.pt
Preprocess the audio clips in : data/train\audio


100%|██████████| 106/106 [00:07<00:00, 13.95it/s]


Preprocess the audio clips in : data/val\audio


100%|██████████| 2/2 [00:00<00:00, 18.82it/s]


## 3. 학습

In [3]:
ddsp_train(args)

 >    exp: exp/combsub-test
 [DDSP Model] Combtooth Subtractive Synthesiser
Load all the data from : data/train


100%|██████████| 106/106 [00:02<00:00, 50.99it/s]


Load all the data from : data/val


100%|██████████| 2/2 [00:00<00:00, 52.34it/s]


--- model size ---
model: 3,607,302
======= start training =======
epoch: 1 |   4/  5 | exp/combsub-test | batch/s: 1.81 | loss: 5.269 | time: 0:00:05.5 | step: 10
epoch: 3 |   4/  5 | exp/combsub-test | batch/s: 20.65 | loss: 4.274 | time: 0:00:06.0 | step: 20
epoch: 5 |   4/  5 | exp/combsub-test | batch/s: 20.44 | loss: 3.112 | time: 0:00:06.5 | step: 30
epoch: 7 |   4/  5 | exp/combsub-test | batch/s: 22.00 | loss: 2.461 | time: 0:00:06.9 | step: 40
epoch: 9 |   4/  5 | exp/combsub-test | batch/s: 18.78 | loss: 1.732 | time: 0:00:07.5 | step: 50
epoch: 11 |   4/  5 | exp/combsub-test | batch/s: 21.19 | loss: 1.331 | time: 0:00:07.9 | step: 60
epoch: 13 |   4/  5 | exp/combsub-test | batch/s: 21.91 | loss: 1.239 | time: 0:00:08.4 | step: 70
epoch: 15 |   4/  5 | exp/combsub-test | batch/s: 19.66 | loss: 1.133 | time: 0:00:08.9 | step: 80
epoch: 17 |   4/  5 | exp/combsub-test | batch/s: 21.53 | loss: 1.318 | time: 0:00:09.4 | step: 90
epoch: 19 |   4/  5 | exp/combsub-test | batch/s

KeyboardInterrupt: 

## 4. 결과물 뽑기

In [13]:
from types import SimpleNamespace

# configure setting
configures = {
    'model_path'            :   'exp/combsub-test/model_best.pt', # 추론에 사용하고자 하는 모델, 바로위에서 학습한 모델을 가져오면댐
    'input'                 :   'data/train/audio/video-0000.wav', # 추론하고자 하는 노래파일의 위치 - 님들이 바꿔야댐 
    'output'                :   'output.wav',  # 결과물 파일의 위치
    'spk_id'                :   '1', 
    'spk_mix_dict'          :   'None', 
    'key'                   :   '0', 
    'enhance'               :   'true' , 
    'pitch_extractor'       :   'crepe' ,
    'f0_min'                :   '50' ,
    'f0_max'                :   '1100',
    'threhold'              :   '-60',
    'enhancer_adaptive_key' :   '0'
}
cmd = SimpleNamespace(**configures)

inference(cmd)

 [DDSP Model] Combtooth Subtractive Synthesiser
 [Loading] exp/combsub-test/model_best.pt
MD5: 44aa2cc3a0a8aa04bde53b9e9b7e729c
Loading pitch curves for input audio from cache directory...
Extracting the volume envelope of the input audio...
 [Encoder Model] HuBERT Soft
 [Loading] pretrain/hubert/hubert-soft-0d54a1f4.pt
Enhancer type: nsf-hifigan
| Load HifiGAN:  pretrain/nsf_hifigan/model
Removing weight norm...
Speaker ID: 1
Cut the input audio into 3 slices


100%|██████████| 3/3 [00:00<00:00,  9.85it/s]
